In [ ]:
from clease.calculator import attach_calculator

In [ ]:
from clease import Concentration, CEBulk, NewStructures

In [ ]:
from clease.montecarlo import Montecarlo

In [ ]:
from clease.tools import update_db

In [ ]:
from ase.calculators.emt import EMT

In [ ]:
from ase.db import connect

# Select concentration

In [ ]:
conc = Concentration(basis_elements=[['Au', 'Cu']])
conc.set_conc_ranges(ranges=[[(0, 0.5), (0.5, 1)]])

# Setup cluster expansion

In [ ]:
setting = CEBulk(
    crystalstructure='fcc',
    a=3.8,
    supercell_factor=27,
    concentration=conc,
    db_name="aucu_bulk.db",
    max_cluster_size=4,
    max_cluster_dia=[6.0, 5.0, 5.0],
    basis_function='polynomial'
)

# Generate structures 

In [ ]:
ns = NewStructures(
    setting=setting, 
    generation_number=0, 
    struct_per_gen=10
)

In [ ]:
ns.generate_initial_pool()

In [ ]:
atoms = connect('aucu_bulk.db').get(id=11).toatoms()

In [ ]:
ns.generate_random_structures(atoms)

# Calculate energies using EMT

In [ ]:
calc = EMT()

In [ ]:
db_name = "aucu_bulk.db"
db = connect(db_name)

In [ ]:
for row in db.select(converged=False):
    atoms = row.toatoms()
    atoms.set_calculator(calc)
    atoms.get_potential_energy()
    update_db(uid_initial=row.id, final_struct=atoms, db_name=db_name)

# Training / Fitting 

In [ ]:
setting = CEBulk(
    crystalstructure='fcc',
    a=3.8,
    supercell_factor=27,
    concentration=conc,
    db_name="aucu_bulk.db",
    max_cluster_size=4,
    max_cluster_dia=[6.0, 5.0, 5.0],
    basis_function='polynomial'
)

# Monte Carlo

In [ ]:
eci = {'c0': -1.0, 'c1_0': 0.1, 'c2_d0000_0_00': -0.2}

In [ ]:
atoms = setting.atoms.copy()*(5, 5, 5)
atoms = attach_calculator(setting, atoms=atoms, eci=eci)

In [ ]:
atoms[0].symbol = 'Cu'
atoms[1].symbol = 'Cu'
atoms[2].symbol = 'Cu'

In [ ]:
T = 500
mc = Montecarlo(atoms, T)
mc.run(steps=1000)

In [ ]:
thermo = mc.get_thermodynamic_quantities()